In [1]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

from os import path
from glob import glob
import pickle

import numpy as np
import pandas as pd
import scipy as sp
import seaborn as sns
import matplotlib as mpl
import matplotlib.pyplot as plt
from matplotlib import cm
from IPython.display import display
from IPython.utils.capture import capture_output
from tqdm.auto import tqdm
with capture_output():
    tqdm.pandas()
import h5py

from ipympl.backend_nbagg import Canvas
Canvas.header_visible.default_value = False

In [2]:
stimulus_categories = {
    'stim_dg_90' : 'complex',
    'stim_dgc_90' : 'simple',
    'stim_flash_90' : 'simple',
    'stim_gabors_90' : 'simple',
    'stim_natural_movie_one_shuffled' : 'shuffled',
    'natural_movies' : 'natural',
    'stim_natural_movie_one' : 'natural',
    'stim_natural_movie_three' : 'natural',
    'spontaneous' : 'spontaneous',
    'stim_sg_90' : 'complex'
}

stim_cat_colors = {
    'spontaneous' : cm.Greys(0.8, 0.8),
    'simple' : cm.Reds(0.8, 0.8),
    'complex' : cm.Blues(0.8, 0.8),
    'natural' : cm.Greens(0.8, 0.8),
    'shuffled' : cm.Purples(0.4, 0.8)
    
}

stim_cat_colors_bg = {
    'spontaneous' : cm.Greys(0.5, 0.5),
    'simple' : cm.Reds(0.5, 0.5),
    'complex' : cm.Blues(0.5, 0.5),
    'natural' : cm.Greens(0.5, 0.5),
    'shuffled' : cm.Purples(0.4, 0.5)
    
}

In [3]:
cnn_models = {
    'VGG16' : '../tiny-blue-dot/differentiation/refactor/CNNs/vgg16_diffn.pkl',
    'ResNet50' : '../tiny-blue-dot/differentiation/refactor/CNNs/resnet_diffn.pkl',
    'InceptionV3' : '../tiny-blue-dot/differentiation/refactor/CNNs/inception_diffn.pkl'
}

differentiation = {
    k : pd.read_pickle(cnn_models[k]) for k in cnn_models.keys()
}

# differentiation = {
#     k : d / d.columns.get_level_values(1).map(np.sqrt) for k, d in differentiation.items()
# }

ylims = {'VGG16': (3291.9816826375727, 4367909.777423014),
 'ResNet50': (253.73407405560303, 72457426.7240913),
 'InceptionV3': (684.8871165012145, 10736165.239076167)}

xlims = {'VGG16': (0.5244677678629754, 20.435278098806037),
 'ResNet50': (0.49113097065560307, 17.424850357749584),
 'InceptionV3': (0.5543381394014095, 13.442327018738055)}

In [4]:
sns.axes_style('whitegrid')

{'figure.facecolor': 'white',
 'axes.labelcolor': '.15',
 'xtick.direction': 'out',
 'ytick.direction': 'out',
 'xtick.color': '.15',
 'ytick.color': '.15',
 'axes.axisbelow': True,
 'grid.linestyle': '-',
 'text.color': '.15',
 'font.family': ['sans-serif'],
 'font.sans-serif': ['Arial',
  'DejaVu Sans',
  'Liberation Sans',
  'Bitstream Vera Sans',
  'sans-serif'],
 'lines.solid_capstyle': 'round',
 'patch.edgecolor': 'w',
 'patch.force_edgecolor': True,
 'image.cmap': 'rocket',
 'xtick.top': False,
 'ytick.right': False,
 'axes.grid': True,
 'axes.facecolor': 'white',
 'axes.edgecolor': '.8',
 'grid.color': '.8',
 'axes.spines.left': True,
 'axes.spines.bottom': True,
 'axes.spines.right': True,
 'axes.spines.top': True,
 'xtick.bottom': False,
 'ytick.left': False}

In [5]:
win, sta = 3, 0.1

with sns.axes_style('whitegrid', rc={'axes.grid': False}):
    f, axes = plt.subplots(3, 1, figsize=(8, 5), tight_layout=True)

for ax, (k, diffn) in zip(axes, differentiation.items()):
    _df = diffn[win][sta].melt().dropna()
    _df = _df[_df.value>1]
    _df['stimulus_categories'] = _df.stimulus.map(stimulus_categories)
#     _df.head()

    flierprops = dict(markerfacecolor=cm.Greys(0.5, 0.5), markeredgecolor='none')

    sns.boxplot(
        x='layer', y='value', hue='stimulus_categories', data=_df,
        ax=ax, fliersize=0.2, linewidth=0.2, whis=1.5,
        palette=stim_cat_colors_bg, width=0.8,
        flierprops=flierprops, showfliers=False
    )
    ax.set_xlim(xlims[k])
    ax.set_ylim(ylims[k])
    ax.legend().set_visible(False)
    ax.set_ylabel(f'{k}\ndifferentiation')
    ax.set_yscale('log')
    ax.set_xlabel('')
    ax.set_xticks([])
    sns.despine(ax=ax)
ax.set_xlabel('CNN layer hierarchy')
axes[0].legend(ncol=4, fontsize=8, frameon=False);
f.savefig('fig_cnn_differentiation.pdf')

Canvas(toolbar=Toolbar(toolitems=[('Home', 'Reset original view', 'home', 'home'), ('Back', 'Back to previous …

In [6]:
ylims = {}
for ax, k in zip(axes, differentiation.keys()):
    ylims[k] = ax.get_ylim()
ylims

{'VGG16': (3291.9816826375727, 4367909.777423014),
 'ResNet50': (253.73407405560303, 72457426.7240913),
 'InceptionV3': (684.8871165012145, 10736165.239076167)}

In [7]:
xlims = {}
for ax, k in zip(axes, differentiation.keys()):
    xlims[k] = ax.get_xlim()
xlims

{'VGG16': (0.5244677678629754, 20.435278098806037),
 'ResNet50': (0.49113097065560307, 17.424850357749584),
 'InceptionV3': (0.5543381394014095, 13.442327018738055)}

# Statistical tests for trends

In [34]:
elb = {
    'VGG16' : 16,
    'ResNet50' : 11,
    'InceptionV3' : 10
}
corrs = {}
for k, diffn in differentiation.items():
    df = diffn[win][sta].stack(0).swaplevel().sort_index()
    df.index = df.index.get_level_values('stimulus').map(stimulus_categories)
    df = df.sort_index()
    corrs[f'{k} (up to {elb[k]})'] = df.groupby('stimulus').apply(
        lambda d: pd.Series(sp.stats.pearsonr(
            *d[range(elb[k])].reset_index(drop=True)
            .unstack().reset_index('layer').values.T
        ), index=['correlation', 'p-value'])
    )
pd.concat(corrs)

correlation       p-value
                       stimulus                           
VGG16 (up to 16)       complex      0.447336  2.043038e-48
                       natural      0.422617  5.353827e-36
                       shuffled     0.604647  2.527264e-17
                       simple       0.252380  2.319659e-22
ResNet50 (up to 11)    complex      0.082606  3.385512e-02
                       natural      0.671780  1.925059e-73
                       shuffled     0.725309  3.225824e-19
                       simple       0.122419  1.127009e-04
InceptionV3 (up to 10) complex      0.486563  5.526303e-37
                       natural      0.486991  3.878880e-31
                       shuffled     0.601209  3.743691e-11
                       simple       0.517061  1.117878e-62

In [49]:
elb = {
    'VGG16' : 16,
    'ResNet50' : 11,
    'InceptionV3' : 10
}
corrs = {}
for k, diffn in differentiation.items():
    df = diffn[win][sta].stack(0).swaplevel().sort_index()
    df.index = df.index.get_level_values('stimulus').map(stimulus_categories)
    df = df.sort_index()
    corrs[f'{k} ({elb[k]} onwards)'] = df.groupby('stimulus').apply(
        lambda d: pd.Series(sp.stats.pearsonr(
            *d[range(elb[k], len(d.columns)-1)].reset_index(drop=True)
            .unstack().reset_index('layer').values.T
        ), index=['correlation', 'p-value'])
    )
pd.concat(corrs)

correlation       p-value
                         stimulus                           
VGG16 (16 onwards)       complex      0.323536  2.982855e-07
                         natural     -0.011031  8.767979e-01
                         shuffled     0.107171  5.104004e-01
                         simple       0.021042  6.907103e-01
ResNet50 (11 onwards)    complex     -0.465983  8.346108e-21
                         natural     -0.677327  1.275326e-41
                         shuffled    -0.839657  5.189094e-17
                         simple      -0.596337  2.623994e-53
InceptionV3 (10 onwards) complex     -0.674080  3.369051e-25
                         natural     -0.752189  1.341123e-28
                         shuffled    -0.944722  4.336741e-15
                         simple      -0.333435  1.965870e-08